In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import math
from skimage.feature import hog
import pickle

In [2]:
def extract_features(images) :
    X = np.empty((0,1152),dtype='i2')
    for image in images:
        fd1, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)
        X = np.append(X,np.reshape(fd1,(1,-1)),axis=0)
    return X

In [3]:
# def denoise_image(images):
#     no_noise = []
# #     no_noise = np.empty(images.shape)
#     for i in range(len(images)):
#         blur = cv2.GaussianBlur(images[i], (5, 5), 0)
#         no_noise.append(blur)
#     return no_noise

In [4]:
def convert_to_binary_labels(labels):
    bin_y = []
    for i in range(0, len(labels)):
        if labels[i] != 0:
            bin_y.append(1)
        else:
            bin_y.append(0)
    return bin_y

In [5]:
def enhance_edges(image):
    return

In [6]:
def split_dataset_into_classes(images):
    class_0, class_1, class_2, class_3, class_4 = []
    return class_0, class_1, class_2, class_3, class_4

In [7]:
X = np.load("Images.npy")
y = np.load("Labels.npy")
# images_test = np.load("")
# labels_test = np.load("")

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=69)

In [9]:
bin_y = convert_to_binary_labels(y_train)
y_test = convert_to_binary_labels(y_test)
print(len(bin_y))

82


In [10]:
X_train = extract_features(X_train)
X_test = extract_features(X_test)
print(X_train.shape)

(82, 1152)


In [11]:
# X_train = denoise_image(X_train)
# print(len(X_train))

In [12]:
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report

In [13]:
EPOCHS = 50
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [14]:
## train data
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = trainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(bin_y))
## test data    
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = testData(torch.FloatTensor(X_test))

In [15]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [21]:
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        # Number of input features is 12.
        self.layer_1 = nn.Linear(1152, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_out = nn.Linear(64, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


model = binaryClassification()
model.to(device)
print(model)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [23]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [24]:
model = binaryClassification()
model.to(device)
print(model)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

binaryClassification(
  (layer_1): Linear(in_features=1152, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [25]:

model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
#         print(X_batch[0])
#         print(y_batch.shape)
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 001: | Loss: 0.66596 | Acc: 58.500
Epoch 002: | Loss: 0.38526 | Acc: 92.000
Epoch 003: | Loss: 0.27406 | Acc: 97.500
Epoch 004: | Loss: 0.24679 | Acc: 97.000
Epoch 005: | Loss: 0.21823 | Acc: 100.000
Epoch 006: | Loss: 0.16334 | Acc: 99.000
Epoch 007: | Loss: 0.14499 | Acc: 100.000
Epoch 008: | Loss: 0.15924 | Acc: 100.000
Epoch 009: | Loss: 0.18924 | Acc: 100.000
Epoch 010: | Loss: 0.11884 | Acc: 100.000
Epoch 011: | Loss: 0.10942 | Acc: 100.000
Epoch 012: | Loss: 0.11100 | Acc: 100.000
Epoch 013: | Loss: 0.13194 | Acc: 100.000
Epoch 014: | Loss: 0.10758 | Acc: 100.000
Epoch 015: | Loss: 0.08935 | Acc: 100.000
Epoch 016: | Loss: 0.09440 | Acc: 97.000
Epoch 017: | Loss: 0.07995 | Acc: 100.000
Epoch 018: | Loss: 0.06046 | Acc: 100.000
Epoch 019: | Loss: 0.09199 | Acc: 100.000
Epoch 020: | Loss: 0.05407 | Acc: 100.000
Epoch 021: | Loss: 0.06018 | Acc: 100.000
Epoch 022: | Loss: 0.05514 | Acc: 100.000
Epoch 023: | Loss: 0.13005 | Acc: 97.000
Epoch 024: | Loss: 0.05551 | Acc: 100.000

In [26]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        print(X_batch[0])
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

tensor([0.3749, 0.2593, 0.4992,  ..., 0.3244, 0.2002, 0.2939])
tensor([0.3856, 0.1389, 0.3071,  ..., 0.3499, 0.1877, 0.1023])
tensor([0.4347, 0.4347, 0.2425,  ..., 0.1606, 0.2639, 0.4493])
tensor([0.3842, 0.2665, 0.3395,  ..., 0.3114, 0.2877, 0.3382])
tensor([0.4044, 0.3643, 0.2719,  ..., 0.2992, 0.1385, 0.4943])
tensor([0.4344, 0.4344, 0.4344,  ..., 0.3536, 0.3536, 0.3536])
tensor([0.3726, 0.1184, 0.2947,  ..., 0.2590, 0.2749, 0.3101])
tensor([0.3814, 0.3657, 0.2690,  ..., 0.4550, 0.4550, 0.1731])
tensor([0.3986, 0.3454, 0.3986,  ..., 0.2596, 0.3822, 0.3822])
tensor([0.0220, 0.0000, 0.0558,  ..., 0.2970, 0.3544, 0.2161])
tensor([0.4767, 0.4767, 0.4767,  ..., 0.3536, 0.3536, 0.3536])
tensor([0.3536, 0.3536, 0.3536,  ..., 0.0700, 0.2139, 0.3486])
tensor([0.1155, 0.0456, 0.2108,  ..., 0.3410, 0.2895, 0.1740])
tensor([0.4153, 0.2826, 0.4153,  ..., 0.3573, 0.2130, 0.3166])
tensor([0.3536, 0.3536, 0.3536,  ..., 0.2873, 0.4662, 0.2557])
tensor([0.1659, 0.0809, 0.1757,  ..., 0.5140, 0.2555, 0

In [27]:
confusion_matrix(y_test, y_pred_list)

array([[ 5,  1],
       [ 5, 17]])